<a href="https://colab.research.google.com/github/ykalathiya-2/AutoGluon/blob/main/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use AutoGluon for Kaggle competitions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/stable/docs/tutorials/tabular/advanced/tabular-kaggle.ipynb)



This tutorial will teach you how to use AutoGluon to become a serious Kaggle competitor without writing lots of code.
We first outline the general steps to use AutoGluon in Kaggle contests. Here, we assume the competition involves tabular data which are stored in one (or more) CSV files.

1) Run Bash command: pip install kaggle

2) Navigate to: https://www.kaggle.com/account and create an account (if necessary).
Then , click on "Create New API Token" and move downloaded file to this location on your machine: `~/.kaggle/kaggle.json`. For troubleshooting, see [Kaggle API instructions](https://www.kaggle.com/docs/api).

3) To download data programmatically: Execute this Bash command in your terminal:

`kaggle competitions download -c [COMPETITION]`

Here, [COMPETITION] should be replaced by the name of the competition you wish to enter.
Alternatively, you can download data manually: Just navigate to website of the Kaggle competition you wish to enter, click "Download All", and accept the competition's terms.

4) If the competition's training data is comprised of multiple CSV files, use [pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) to properly merge/join them into a single data table where rows = training examples, columns = features.

5) Run autogluon `fit()` on the resulting data table.

6) Load the test dataset from competition (again making the necessary merges/joins to ensure it is in the exact same format as the training data table), and then call autogluon `predict()`.  Subsequently use [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) to load the competition's `sample_submission.csv` file into a DataFrame, put the AutoGluon predictions in the right column of this DataFrame, and finally save it as a CSV file via [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html). If the competition does not offer a sample submission file, you will need to create the submission file yourself by appropriately reformatting AutoGluon's test predictions.

7) Submit your predictions via Bash command:

`kaggle competitions submit -c [COMPETITION] -f [FILE] -m ["MESSAGE"]`

Here, [COMPETITION] again is the competition's name, [FILE] is the name of the CSV file you created with your predictions, and ["MESSAGE"] is a string message you want to record with this submitted entry. Alternatively, you can  manually upload your file of predictions on the competition website.

8) Finally, navigate to competition leaderboard website to see how well your submission performed!
It may take time for your submission to appear.



Below, we demonstrate how to do steps (4)-(6) in Python for a specific Kaggle competition: [ieee-fraud-detection](https://www.kaggle.com/c/ieee-fraud-detection/).
This means you'll need to run the above steps with `[COMPETITION]` replaced by `ieee-fraud-detection` in each command.  Here, we assume you've already completed steps (1)-(3) and the data CSV files are available on your computer. To begin step (4), we first load the competition's training data into Python:

In [1]:
!unzip ieee-fraud-detection.zip

Archive:  ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


In [2]:
!pip install --quiet autogluon

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 8.0 MB/s eta 0:00:00
 

In [3]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models

train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

Since the training data for this competition is comprised of multiple CSV files, we just first join them into a single large table (with rows = examples, columns = features) before applying AutoGluon:

In [4]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

```
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
```


Note that a left-join on the `TransactionID` key happened to be most appropriate for this Kaggle competition, but for others involving multiple training data files, you will likely need to use a different join strategy (always consider this very carefully). Now that all our training data resides within a single table, we can apply AutoGluon. Below, we specify the `presets` argument to maximize AutoGluon's predictive accuracy which usually requires that you run `fit()` with longer time limits (3600s below should likely be increased in your run):

In [ ]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='best_quality', time_limit=3600
)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Sep  6 09:54:41 UTC 2025
CPU Count:          8
GPU Count:          1
Memory Avail:       43.94 GB / 50.99 GB (86.2%)
Disk Space Avail:   188.38 GB / 235.68 GB (79.9%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'num_bag_sets': 1}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
             'holdout_data': None,
 

(_ray_fit pid=7150) [50]	valid_set's binary_logloss: 0.0947332
(_ray_fit pid=7151) [100]	valid_set's binary_logloss: 0.0874943 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=7150) [150]	valid_set's binary_logloss: 0.0811864 [repeated 2x across cluster]
(_ray_fit pid=7151) [200]	valid_set's binary_logloss: 0.0794327 [repeated 2x across cluster]
(_ray_fit pid=7151) [250]	valid_set's binary_logloss: 0.0772054 [repeated 2x across cluster]
(_ray_fit pid=7151) [300]	valid_set's binary_logloss: 0.0752429 [repeated 3x across cluster]
(_ray_fit pid=7151) [350]	valid_set's binary_logloss: 0.0738669
(_ray_fit pid=7150) [350]	valid_set's binary_logloss: 0.0715076
(_ray_fit pid=7151) [450]	valid_set's binary_logloss: 0.0709354 [repeated 3x across cluster]
(_ray_fit pid=7151) [500]	valid_set's 

(_ray_fit pid=7150) 	Ran out of time, early stopping on iteration 933. Best iteration is:
(_ray_fit pid=7150) 	[933]	valid_set's binary_logloss: 0.0609634
(_ray_fit pid=7151) 	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}
(_ray_fit pid=7150) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F1/model.pkl
(_ray_fit pid=7805) 	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}
(_ray_fit pid=7151) 	Ran out of time, early stopping on iteration 969. Best iteration is:
(_ray_fit pid=7151) 	[969]	valid_set's binary_logloss: 0.0632025
(_ray_fit pid=7151) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F2/model.pkl


(_ray_fit pid=7805) [50]	valid_set's binary_logloss: 0.0972117
(_ray_fit pid=7806) [50]	valid_set's binary_logloss: 0.0919926
(_ray_fit pid=7805) [100]	valid_set's binary_logloss: 0.088068
(_ray_fit pid=7806) [100]	valid_set's binary_logloss: 0.0834436
(_ray_fit pid=7805) [150]	valid_set's binary_logloss: 0.0833867
(_ray_fit pid=7806) [150]	valid_set's binary_logloss: 0.07891
(_ray_fit pid=7805) [200]	valid_set's binary_logloss: 0.0802051
(_ray_fit pid=7806) [200]	valid_set's binary_logloss: 0.0761137
(_ray_fit pid=7806) [250]	valid_set's binary_logloss: 0.0738514 [repeated 2x across cluster]
(_ray_fit pid=7805) [300]	valid_set's binary_logloss: 0.0757618
(_ray_fit pid=7806) [300]	valid_set's binary_logloss: 0.071915
(_ray_fit pid=7806) [400]	valid_set's binary_logloss: 0.0689046 [repeated 3x across cluster]
(_ray_fit pid=7806) [450]	valid_set's binary_logloss: 0.067777 [repeated 2x across cluster]
(_ray_fit pid=7806) [550]	valid_set's binary_logloss: 0.0658066 [repeated 4x across clus

(_ray_fit pid=7805) 	Ran out of time, early stopping on iteration 895. Best iteration is:
(_ray_fit pid=7805) 	[895]	valid_set's binary_logloss: 0.0642926
(_ray_fit pid=7806) 	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}
(_ray_fit pid=7805) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F3/model.pkl
(_ray_fit pid=8451) 	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}
(_ray_fit pid=7806) 	Ran out of time, early stopping on iteration 918. Best iteration is:
(_ray_fit pid=7806) 	[918]	valid_set's binary_logloss: 0.0602902
(_ray_fit pid=7806) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F4/model.pkl


(_ray_fit pid=8451) [50]	valid_set's binary_logloss: 0.0954958
(_ray_fit pid=8452) [50]	valid_set's binary_logloss: 0.0947801
(_ray_fit pid=8451) [100]	valid_set's binary_logloss: 0.0869681
(_ray_fit pid=8452) [100]	valid_set's binary_logloss: 0.085936
(_ray_fit pid=8451) [150]	valid_set's binary_logloss: 0.0821951
(_ray_fit pid=8452) [150]	valid_set's binary_logloss: 0.0804302
(_ray_fit pid=8452) [200]	valid_set's binary_logloss: 0.0770633 [repeated 2x across cluster]
(_ray_fit pid=8452) [250]	valid_set's binary_logloss: 0.0746041
(_ray_fit pid=8451) [250]	valid_set's binary_logloss: 0.0765972
(_ray_fit pid=8451) [300]	valid_set's binary_logloss: 0.0746635 [repeated 2x across cluster]
(_ray_fit pid=8452) [400]	valid_set's binary_logloss: 0.0695193 [repeated 3x across cluster]
(_ray_fit pid=8452) [500]	valid_set's binary_logloss: 0.0670411 [repeated 4x across cluster]
(_ray_fit pid=8452) [550]	valid_set's binary_logloss: 0.0660105 [repeated 2x across cluster]
(_ray_fit pid=8451) [600]	

(_ray_fit pid=8451) 	Ran out of time, early stopping on iteration 964. Best iteration is:
(_ray_fit pid=8451) 	[964]	valid_set's binary_logloss: 0.0624964
(_ray_fit pid=8452) 	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}
(_ray_fit pid=8451) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F5/model.pkl
(_ray_fit pid=9097) 	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True}
(_ray_fit pid=8452) 	Ran out of time, early stopping on iteration 966. Best iteration is:
(_ray_fit pid=8452) 	[966]	valid_set's binary_logloss: 0.0599292
(_ray_fit pid=8452) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F6/model.pkl


(_ray_fit pid=9096) [50]	valid_set's binary_logloss: 0.0953804 [repeated 3x across cluster]
(_ray_fit pid=9097) [100]	valid_set's binary_logloss: 0.0859983 [repeated 2x across cluster]
(_ray_fit pid=9097) [150]	valid_set's binary_logloss: 0.0813892 [repeated 2x across cluster]
(_ray_fit pid=9096) [200]	valid_set's binary_logloss: 0.0786712 [repeated 2x across cluster]
(_ray_fit pid=9096) [250]	valid_set's binary_logloss: 0.0763081 [repeated 2x across cluster]
(_ray_fit pid=9097) [300]	valid_set's binary_logloss: 0.0731009 [repeated 3x across cluster]
(_ray_fit pid=9096) [350]	valid_set's binary_logloss: 0.0727327
(_ray_fit pid=9097) [350]	valid_set's binary_logloss: 0.0714909


```
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='best_quality', time_limit=3600
)

results = predictor.fit_summary()
```


Now, we use the trained AutoGluon Predictor to make predictions on the competition's test data. It is imperative that multiple test data files are joined together in the exact same manner as the training data. Because this competition is evaluated based on the AUC (Area under the ROC curve) metric, we ask AutoGluon for predicted class-probabilities rather than class predictions. In general, when to use `predict` vs `predict_proba` will depend on the particular competition.

In [ ]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

```
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities
```


When submitting predicted probabilities for classification competitions, it is imperative these correspond to the same class expected by Kaggle. For binary classification tasks, you can see which class AutoGluon's predicted probabilities correspond to via:

In [ ]:
predictor.positive_class

```
predictor.positive_class
```


For multiclass classification tasks, you can see which classes AutoGluon's predicted probabilities correspond to via:

In [ ]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

```
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output
```


Now, let's get prediction probabilities for the entire test data, while only getting the positive class predictions by specifying:

In [ ]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

```
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)
```


Now that we have made a prediction for each row in the test dataset, we can submit these predictions to Kaggle. Most Kaggle competitions provide a sample submission file, in which you can simply overwrite the sample predictions with your own as we do below:

In [ ]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)

```
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)
```


We have now completed steps (4)-(6) from the top of this tutorial. To submit your predictions to Kaggle, you can run the following command in your terminal (from the appropriate directory):

`kaggle competitions submit -c ieee-fraud-detection -f sample_submission.csv -m "my first submission"`

You can now play with different `fit()` arguments and feature-engineering techniques to try and maximize the rank of your submissions in the Kaggle Leaderboard!


**Tips to maximize predictive performance:**

   - Be sure to specify the appropriate evaluation metric if one is specified on the competition website! If you are unsure which metric is best, then simply do not specify this argument when invoking `fit()`; AutoGluon should still produce high-quality models by automatically inferring which metric to use.

   - If the training examples are time-based and the competition test examples come from future data, we recommend you reserve the most recently-collected training examples as a separate validation dataset passed to `fit()`. Otherwise, you do not need to specify a validation set yourself and AutoGluon will automatically partition the competition training data into its own training/validation sets.

   - Beyond simply specifying `presets = 'best_quality'`, you may play with more advanced `fit()` arguments such as: `num_bag_folds`, `num_stack_levels`, `num_bag_sets`, `hyperparameter_tune_kwargs`, `hyperparameters`, `refit_full`. However we recommend spending most of your time on feature-engineering and just specify `presets = 'best_quality'` inside the call to `fit()`.


**Troubleshooting:**

- Check that you have the right user-permissions on your computer to access the data files downloaded from Kaggle.

- For issues downloading Kaggle data or submitting predictions, check your Kaggle account setup and the [Kaggle FAQ](https://www.kaggle.com/general/14438).